In [1]:
import hope
hope.config.optimize = True
hope.config.verbose = True
hope.config.keeptemp = True
import numba
import numpy as np
import numexpr as ne

from util import perf_comp_data
from native_util import load
%load_ext Cython
%load_ext version_information
%version_information numpy, Cython, numba, hope, numexpr

Software versions
Python 3.5.1 64bit [GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]
IPython 6.1.0
OS Darwin 15.6.0 x86_64 i386 64bit
numpy 1.13.1
Cython 0.26
numba 0.34.0
hope 0.6.1
numexpr 2.6.2
Mon Sep 04 15:28:09 2017 CEST

In [5]:
def poly(res, arg):
    res[:] = np.sin(arg)**2 + (arg**3 + arg**2 - arg - 1)/(arg**2 + 2*arg + 1) + np.cos(arg)**2
hope_poly = hope.jit(poly)
numba_poly = numba.jit(poly, nopython=False)

native_poly_mod = load("poly")
native_poly = native_poly_mod.run


running build_ext
building 'poly' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include -I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11 -Wno-unreachable-code'
clang: ././src/poly.cpp
/usr/bin/clang++ -bundle -undefined dynamic_lookup -arch i386 -arch x86_64 -g ./src/poly.o -o ./poly.cpython-35m-darwin.so



In [9]:
%%cython -a

cimport cython
from libc.math cimport sin
from libc.math cimport cos
from libc.math cimport pow

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def cython_poly(double[:] res, double[:] arg):
    cdef int i
    cdef double i_arg
    for i in range(len(arg)):
        i_arg = arg[i]
        res[i] = pow(sin(i_arg),2) + (pow(i_arg,3) + pow(i_arg,2) - i_arg - 1)/(pow(i_arg,2) + 2*i_arg + 1) + pow(cos(i_arg),2)



building '_cython_magic_c56f3cd16e138e603a8a648f0cd9159f' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c'
clang: /Users/uweschmitt/.ipython/cython/_cython_magic_c56f3cd16e138e603a8a648f0cd9159f.c
/usr/bin/clang -bundle -undefined dynamic_lookup -arch i386 -arch x86_64 -g /Users/uweschmitt/.ipython/cython/Users/uweschmitt/.ipython/cython/_cython_magic_c56f3cd16e138e603a8a648f0cd9159f.o -o /Users/uweschmitt/.ipython/cython/_cython_magic_c56f3cd16e138e603a8a648f0cd9159f.cpython-35m-darwin.so


In [10]:
%%cython
cimport cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def cython_numpy_poly(np.ndarray[np.double_t, ndim=1] res, np.ndarray[np.double_t, ndim=1] arg):
    res[:] = np.sin(arg)**2 + (arg**3 + arg**2 - arg - 1)/(arg**2 + 2*arg + 1) + np.cos(arg)**2


building '_cython_magic_2c4125c086131196c2c8dcbbb7f86ef5' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include -I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c'
clang: /Users/uweschmitt/.ipython/cython/_cython_magic_2c4125c086131196c2c8dcbbb7f86ef5.c
In file included from /Users/uweschmitt/.ipython/cython/_cython_magic_2c4125c086131196c2c8dcbbb7f86ef5.c:495:
In file included from /Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include/numpy/ndarrayobject.h:18:
In file included from /Users/uweschmitt/Projects/hope/venv3/

In [11]:
# NumExpr version

import numexpr as ne

def numexpr_poly(res, arg):
    res[:] = ne.evaluate("sin(arg)**2 + (arg**3 + arg**2 - arg - 1)/(arg**2 + 2*arg + 1) + cos(arg)**2")


In [12]:
arg = np.random.random(50000)
res = np.empty_like(arg)

res1 = np.empty_like(arg)
res2 = np.empty_like(arg)
res3 = np.empty_like(arg)
res4 = np.empty_like(arg)
res5 = np.empty_like(arg)

poly(res1, arg)
hope_poly(res2, arg)
numba_poly(res3, arg)
native_poly(res4, arg)
numexpr_poly(res5, arg)

assert np.allclose(res1, res2)
assert np.allclose(res1, res3)
assert np.allclose(res1, res4)
assert np.allclose(res1, res5)


poly(float64^1 res, float64^1 arg)
	(:res@0) {
		res.d[:res@0] = (((numpy.sin(arg.d[:arg@0]) ** 2.J) + (((((arg.d[:arg@0] ** 3.J) + (arg.d[:arg@0] ** 2.J)) - arg.d[:arg@0]) - 1.J) / (((arg.d[:arg@0] ** 2.J) + (2.J * arg.d[:arg@0])) + 1.J))) + (numpy.cos(arg.d[:arg@0]) ** 2.J))
	}

Compiling following functions:
poly(float64^1 res, float64^1 arg)
running build_ext
building 'poly_332b2c0332e8513cfd35081ce10b11e6c57ab0ccac824978e78990c6_0' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -arch i386 -arch x86_64 -g

compile options: '-I/Users/uweschmitt/Projects/hope/venv3/lib/python3.5/site-packages/numpy/core/include -I/Users/uweschmitt/Projects/hope/venv3/include -I/Library/Frameworks/Python.framework/Versions/3.5/include/python3.5m -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11 -Wno-unreachable-code'
clang: /var/folders/k8/zfp7dvcs1m326gz1brql1tv80000gn/

In [14]:
print("python")
%timeit poly(res, arg)
print("hope")
%timeit hope_poly(res, arg)
print("numba")
%timeit numba_poly(res, arg)
print("cython")
%timeit cython_poly(res, arg)
print("cython numpy")
%timeit cython_numpy_poly(res, arg)
print("native")
%timeit native_poly(res, arg)
print("numexpr (1)")
ne.set_num_threads(1)
%timeit numexpr_poly(res, arg)
print("numexpr ({0})".format(ne.detect_number_of_cores()))
ne.set_num_threads(ne.detect_number_of_cores())
%timeit numexpr_poly(res, arg)



python
2.73 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
hope
28.6 µs ± 686 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
numba
697 µs ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
cython
2.23 ms ± 54.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
cython numpy
3.01 ms ± 60.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
native
2.02 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
numexpr (1)
1.56 ms ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
numexpr (8)
435 µs ± 28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
ne.set_num_threads(8)
perf_comp_data(["poly", 
                "hope_poly", 
                "numba_poly", 
                "cython_poly", 
                "cython_numpy_poly", 
                "native_poly",
                "numexpr_poly"], 
               7*["res, arg"], rep=100)

function: hope_poly           , av. time sec:   0.00005208, min. time sec:   0.00002532, relative:       1.0
function: numexpr_poly        , av. time sec:   0.00045749, min. time sec:   0.00039542, relative:       8.8
function: numba_poly          , av. time sec:   0.00063641, min. time sec:   0.00063479, relative:      12.2
function: cython_poly         , av. time sec:   0.00190999, min. time sec:   0.00176367, relative:      36.7
function: native_poly         , av. time sec:   0.00193384, min. time sec:   0.00189017, relative:      37.1
function: poly                , av. time sec:   0.00260737, min. time sec:   0.00242636, relative:      50.1
function: cython_numpy_poly   , av. time sec:   0.00261038, min. time sec:   0.00245425, relative:      50.1
